# Example JsonSchema

In [1]:
import pathlib
import json_ntv
from json_ntv.namespace import from_file
from json_ntv.ntv_schema import ntv_validate
from json_ntv import Ntv
from jsonschema import validate

file = pathlib.Path(json_ntv.__file__).parent.parent / "RFC" / "NTV_NTVschema_namespace.ini"
from_file(file, '$NTVschema.')

## JSONschema

In [2]:
json_test = {'family' : 'white', 
             'children': [15, 
                          24, 
                          {'walter jr': None}, 
                          {'judith': 22}]}

json_schema = {'properties': {'family':{
                                    'maxLength':10},
                              'children':{
                                    'maxItems': 4,
                                    'items': {
                                        'maximum': 25, 
                                        'propertyNames': {'maxLength':10},
                                        'properties':{
                                            'judith': {'maximum': 23}
                                        }}}}}


validate(json_test, json_schema)

`properties` and `prefixItems` are used to navigate in the Json tree

In [3]:
list_json_test = [
 ['',                   '',         ''     , 2],
 ['/family',            'family',   'white', 1],
 ['/children',          'children', ''     , 4],
 ['/children/0',        '',         15     , 1],
 ['/children/1',        '',         21     , 1],
 ['/children/2',        '',         ''     , 1],
 ['/children/2/simon',  'walter jr',None   , 1],
 ['/children/3',        '',         ''     , 1],
 ['/children/3/judith', 'judith',   22     , 1] ]

list_json_schema = [
 ['/family',     {'maxLength':10}],
 ['/children',   {'maxItems': 4, 'items': {'maximum': 25, 'propertyNames': {'maxLength':10}}}] ]

## NTVschema

In [4]:
ntv_test = {'family:string' : 'white', 
            'children': [{':int': 15}, 
                         21, 
                         {'walter jr':None}, 
                         {'judith': 22}]}

ntv_schema = {'test::$NTVschema.':
                  {'properties.': {
                          'family': {
                                 ':maxLength': 5}, #no_type
                           'children':{  
                                 ':maxItems': 5, #no_type
                                 'items.': {
                                   ':maximum': 25, # no_type, no_name
                                   'name.': {':maxLength': 10}}
                           }}}}
print(ntv_validate(Ntv.obj(ntv_test), ntv_schema['test::$NTVschema.'], mode=1), '\n')

  validate :  
  validate :  /family:string
  validate :  /children
  validate :  /children/0
  validate :  /children/1
  validate :  /children/2
  validate :  /children/3
True 



In [5]:
ntv_test2 = {
      'example family::org.' : {
             'name': 'white',
             'House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets::':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_schema3  = {'test::$NTVschema.':{
                    'properties.': {
                            'House.': {
                                'properties.':{
                                    'location':{
                                        'type.': {':enum': ['point', 'geojson', 'org.Place.address']}},
                                    'arrival': {
                                        'type.': {':enum': ['year', 'date', 'datetime']}}}},
                            'childrens':{  
                                 ':maxItems': 5,
                                 'items.': {
                                     'properties.':{
                                         'age':{
                                             ':maximum': 25,
                                             'type.': {':enum': ['json', 'int']},
                                             }}}},
                            'pets':{
                               'prefixItems.':[
                                   {'type.': {':const':'$dog'}}],
                               'items.':{
                                   'type.': {':enum': ['$dog', '$cat', '$mouse']}},
                               'properties.': {
                                   ':$mouse': {':maximum': 10 }}
                               }
                               }
                           }}
print(ntv_validate(Ntv.obj(ntv_test2), ntv_schema3['test::$NTVschema.'], mode=1))

  validate :  example family::org.
  validate :  example family::org./House.
  validate :  example family::org./House./location:point
  validate :  example family::org./House./arrival:year
  validate :  example family::org./childrens::Person.
  validate :  example family::org./childrens::Person./0
  validate :  example family::org./childrens::Person./0
  validate :  example family::org./childrens::Person./1
  validate :  example family::org./childrens::Person./2
  validate :  example family::org./childrens::Person./2/age
  validate :  example family::org./pets
  validate :  example family::org./pets/:$mouse
  validate :  example family::org./pets/:$dog
  validate :  example family::org./pets/:$dog
  validate :  example family::org./pets/:$cat
  validate :  example family::org./pets/:$mouse
True


In [17]:
ntv_test2 = {
      'example family::org.' : {
             'name': 'white',
             'House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_test3 = {
      'example family' : {
             'name': 'white',
             'org.House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:org.Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_schema2  = {'test::$NTVschema.':{
                    'name.' : {
                           ':maxLength': 50},
                    'properties.': {
                            'org.House.': {
                                'properties.':{
                                    'location':{
                                        'type.': {':enum': ['point', 'geojson', 'org.Place.address']}},
                                    'arrival': {
                                        'type.': {':enum': ['year', 'date', 'datetime']}}}},
                            'childrens':{  
                                 ':maxItems': 5,
                                 'items.': {
                                     'properties.':{
                                         'age':{
                                             ':maximum': 25,
                                             'type.': {':enum': ['json', 'int']},
                                             }}}},
                            'pets':{
                               'prefixItems.':[
                                   {'type.': {':const':'$dog'}}],
                               'items.':{
                                   'type.': {':enum': ['$dog', '$cat', '$mouse']}},
                               'properties': {
                                   ':$mouse': {':maximum': 10 }}
                               }
                           }}}

ntv_schema3  = {'test::$NTVschema.':{
                    'properties.': {
                            'org.House.': {
                                'properties.':{
                                    'location':{
                                        'type.': {':enum': ['point', 'geojson', 'org.Place.address']}},
                                    'arrival': {
                                        'type.': {':enum': ['year', 'date', 'datetime']}}}},
                            'childrens':{  
                                 ':maxItems': 5,
                                 'items.': {
                                     'properties.':{
                                         'age':{
                                             ':maximum': 25,
                                             'type.': {':enum': ['json', 'int']},
                                             }}}},
                            'pets':{
                               'prefixItems.':[
                                   {'type.': {':const':'$dog'}}],
                               'items.':{
                                   'type.': {':enum': ['$dog', '$cat', '$mouse']}},
                               'properties.': {
                                   ':$mouse': {':maximum': 10 }}
                               }
                           }}}
print(ntv_validate(Ntv.obj(ntv_test3), ntv_schema2['test::$NTVschema.'], mode=3))

  validate :  example family
     name  :  example family {'maxLength': 50}
  validate :  example family/org.House.
  validate :  example family/org.House./location:point
     type  :  point {'enum': ['point', 'geojson', 'org.Place.address']}
  validate :  example family/org.House./arrival:year
     type  :  year {'enum': ['year', 'date', 'datetime']}
  validate :  example family/childrens::org.Person.
     value  :  [{"age": 21}, {":org.Person.givenName": "walter jr"}, {"org.Person.": {"age": 22, ":givenName": "judith"}}] {'maxItems': 5}
  validate :  example family/childrens::org.Person./0
  validate :  example family/childrens::org.Person./0
     value  :  21 {'maximum': 25}
     type  :  json {'enum': ['json', 'int']}
  validate :  example family/childrens::org.Person./1
  validate :  example family/childrens::org.Person./2
  validate :  example family/childrens::org.Person./2/age
     value  :  22 {'maximum': 25}
     type  :  json {'enum': ['json', 'int']}
  validate :  example f

In [7]:
ntv_test2 = {
      'example family::org.' : {
             'name': 'white',
             'House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_test3 = {
      'example family' : {
             'name:string': 'white',
             'org.House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:org.Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_schema2  = {':sch':{
                    'name:sch' : {':maxLength': 50},
                    'org.House.:sch': {
                        'location:sch':{
                            ':type': {':enum': ['point', 'geojson', 'org.Place.address']}},
                        'arrival:sch': {
                            ':type': {':enum': ['year', 'date', 'datetime']}},
                        'childrens:sch':{  
                            ':maxItems': 5,
                            ':items': {
                                'age:sch':{
                                    ':maximum': 25,
                                    ':type': {':enum': ['json', 'int']}
                                             }},
                            'pets:sch':{
                               '0:sch':{
                                   ':type': {':const':'$dog'}},
                               ':items':{
                                   ':type': {':enum': ['$dog', '$cat', '$mouse']}},
                               ':$mouse:sch': {':maximum': 10 }}
                               }
                            }
                           }}

ntv_schema3  = {':sch':{
                    '/name:string' : {':maxLength': 50},
                    '/org.House.': {
                        '/location':{':type': {':enum': ['point', 'geojson', 'org.Place.address']}},
                        '/arrival': {':type': {':enum': ['year', 'date', 'datetime']}},
                        '/childrens':{  
                            ':maxItems': 5,
                            ':items': {
                                '/age':{
                                    ':maximum': 25,
                                    ':type': {':enum': ['json', 'int']}
                                             }},
                            '/pets':{
                               '/0':{
                                   ':type': {':const':'$dog'}},
                               ':items':{
                                   ':type': {':enum': ['$dog', '$cat', '$mouse']}},
                               '/:$mouse': {':maximum': 10 }}
                               }
                            }
                           }}

ntv_schema4  = {':sch':{
                    'name:string' : {':maxLength': 50},
                    'org.House.': {
                        'location':{':type': {':enum': ['point', 'geojson', 'org.Place.address']}},
                        'arrival': {':type': {':enum': ['year', 'date', 'datetime']}},
                        'childrens':{  
                            ':maxItems': 5,
                            ':items': {
                                'age':{
                                    ':maximum': 25,
                                    ':type': {':enum': ['json', 'int']}
                                             }},
                            'pets':{
                               '0':{
                                   ':type': {':const':'$dog'}},
                               ':items':{
                                   ':type': {':enum': ['$dog', '$cat', '$mouse']}},
                               ':$mouse': {':maximum': 10 }}
                               }
                            }
                           }}

print(ntv_validate(Ntv.obj(ntv_test2), ntv_schema3, mode=1))

  validate :  example family::org.
True


In [8]:
print(Ntv.obj(ntv_test2).name)
print(Ntv.obj(ntv_test2).pointer())
print(Ntv.obj({'org.':{':$dog': {'hector': 10}, ':$cat': {'tom': 5},':$mouse': 3}})[2].pointer())

example family
example family::org.
org./:$mouse


In [9]:
list_ntv_test = [
 ['',                   '',         ''     , None    , 2],
 ['/family',            'family',   'thomy', 'string', 1],
 ['/children',          'children', ''     , None    , 4],
 ['/children/0',        '',         15     , 'int'   , 1],
 ['/children/1',        '',         21     , 'json'  , 1],
 ['/children/simon',    'simon',    None   , 'json'  , 1],
 ['/children/judith',   'judith',   22     , 'json'  , 1] ]

list_ntv_schema = [
 ['/family',     None,          {':maxLength': 10 }], # ntv_value
 ['/children',   None,          {':maxItems': 4}], # ntv_value
 ['/children',   'items.',      {':maximum': 25}], # ntv_value
 ['/children',   'items.name.', {':maxLength': 10}] ] #ntv_name

list_ntv_schema2 = [
 ['/family',     None,          {':maxLength': 10 }  ],  # ntv_value
 ['/family',     'type.',       {':const': 'string'} ], # ntv_type
 ['/children',   None,          {':maxItems': 4}], # ntv_value
 ['/children',   'items.',      {':maximum': 25} ], # ntv_value
 ['/children',   'items.type.', {':enum': ['json', 'int']} ], # ntv_type
 ['/children',   'items.name.', {':maxLength': 10} ], # ntv_name
 ['/children/0', 'type.',       {':const':'int'}] ] # ntv_type

## Ntv Pointer

In [10]:
ntv_sch2 = Ntv.obj(ntv_schema2)
print(ntv_sch2['properties.']['family']['type.'])
print(ntv_sch2['#test/properties./family/type.'])

print(ntv_sch2['properties.']['family'])
print(ntv_sch2[0][0])
print(ntv_sch2['#0/0/0'])
print(ntv_sch2['#test/0/0'])
print(ntv_sch2['#test/properties./0'])
print(ntv_sch2['#test/properties./family'])

DatatypeError: sch is not defined in 

## JsonSchema keywords


### type
Type of a value 

* schema applicable to NTVvalue
```json
{ "type": ["number", "string"] } check if the NTVvalue is a number or a string
```

### String type
length, format, regex 

* schema applicable to NTVvalue 
```json
{ "type": "string", "minLength": 2, "maxLength": 3 } check the length of the NTVvalue
```


### Number type
multiples, range 

* schema applicable to NTVvalue
```json
{ "type": "number", "multipleOf" : 10 } check if the NTVvalue is a multiple of 10
```


### Object : properties
Schema for a value defined by a name 

* schema applicable to the NTVvalue of the NTV entity defined by his relative NTV-pointer 
```json
{ "properties": { "street_name": value_schema } } check the value_schema for the NTVvalue of the NTV entity defined by "street_name" (relative NTV-pointer)
 ```


### Object : patternProperties
Schema for values defined by a name matches a pattern 

* schema applicable to the NTVvalue of the NTV entity whose relative NTV-pointer matches a pattern.
```json
{ "patternProperties": { "^S_": pat_schema } } check the pat_schema for the NTVvalue of the NTV entity whose relative NTV-pointer matches "^S_"
 ```


### Object : additionalProperties
Schema for values whose names are not listed in the properties or patternProperties 

* schema applicable to the NTVvalue of the NTV entity not listed in the properties or patternProperties.
```json
{ "additionalProperties": add_schema } } check the add_schema for the NTVvalue of the NTV entity not listed in the properties or patternProperties.
 ```


### Object : propertyNames
Schema for names 

* schema applicable to the NTVname of the NTV entities 
```json
{ "propertyNames": names_schema } check the names_schema for the NTVnames of the NTV entity 
 ```


### Object : size
Number of properties 

* Number of NTV entities included in the NTV entity
```json
{ "minProperties": 2, "maxProperties": 3 } check the number of NTV entities 
 ```


### Object : other
Required : applicable,

unevaluatedProperties : idem as additionalProperties

### Array : items
Schema for values included in an array 

* schema applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "items": items_schema } } check the items_schema for the NTVvalue of the NTV entities included in the NTV entity.
 ```


### Array : uniqueItems
Schema to ensure that each of the items in an array is unique 

* schema applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "uniqueItems": true } } check the uniqueness of the NTVvalue of the NTV entities.
 ```


### Array : prefixItems
Array of schemas for values included in an array 

* schemas applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "prefixItems": [ item_schema ] } 
check the item_schema for the NTVvalue of the corresponding NTV entity included in the NTV entity.
 ```


### Array : contains, maxContains, minContains
schema for at least one (or number as defined with maxContains or minContains) value included in an array 

* schema applicable to the NTVvalues of the NTV entities included in the NTV entity 
```json
{ "contains": cont_schema, "minContains": 2, "maxContains": 3 } 
check if the cont_schema is valid for 2 or 3 NTVvalue of the NTV entity included.
 ```


### Array : other
unevaluatedItems : applies to any NTVvalues not evaluated by an items, prefixItems, or contains keyword.

minItems, maxItems : equivalent to minProperties, maxProperties


## NTV extension

### xxproperties
"pointer:part": schema

* pointer : relative NTV pointer (integer, NTVname, NTVtype)
* part : name, type or value (default)

### propertyNames 
equivalent to items:name

### required
list of pointers

### xxitems
"xxitems:part": schema

### xxcontains
"xxcontains:part": schema